In [1]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import get_cosine_schedule_with_warmup

sys.path.append("../../run/")
sys.path.append("../../")
from experimet_config import Config
from src.utils.utils import set_seed
from src.datasets.latent_navigation_dataset import GeometryNavigationDataset
from src.models.regressor import Regressor
from src.models.latent_walker import WalkMlpMultiW
from src.models.latent_walker import WalkEffKANMulti
from src.models.deepsdf import DeepSDF
from src.utils.metric import regressor_criterion

In [2]:
set_seed(42)
cfg = Config()

# loading geometry columns
geometry_columns = cfg.geometry_attribute

# loading training data
merge_df = pd.read_csv("../../data/table/merged.csv").dropna().reset_index(drop=True)
merge_df = merge_df[~merge_df["folder_name"].isin(cfg.noise_data)][
    ["file_name", "folder_name"]
]

geometry_df = (
    pd.read_excel("../../data/table/geometry.xlsx", index_col=0)
    .T.reset_index()
    .rename(columns={"index": "file_name"})
)

geometry_df["基調ライン基点長さ"] = geometry_df["基調ライン基点長さ"] * geometry_df["ホイールベース"]
geometry_df["前輪-Lの長さ"] = geometry_df["前輪-Lの長さ"] * geometry_df["ホイールベース"]
geometry_df["ノーズ高さ"] = geometry_df["ノーズ高さ"] * geometry_df["全高"]
geometry_df["ノーズスラント量"] = geometry_df["ノーズスラント量"] * geometry_df["フード長さ"]
geometry_df["基調ライン基点高さ"] = geometry_df["基調ライン基点高さ"] * geometry_df["全高"]
geometry_df["ベルトライン高さ"] = geometry_df["ベルトライン高さ"] * geometry_df["全高"]
geometry_df["フロントバンパー下端高さ"] = geometry_df["フロントバンパー下端高さ"] * geometry_df["全高"]
geometry_df["リアバンパー下端高さ"] = geometry_df["リアバンパー下端高さ"] * geometry_df["全高"]
geometry_df["サイドシル下端高さ"] = geometry_df["サイドシル下端高さ"] * geometry_df["全高"]
geometry_df["キャビン幅"] = geometry_df["キャビン幅"] * geometry_df["全幅"] * geometry_df["全高"]
geometry_df["ルーフ幅"] = geometry_df["ルーフ幅"] * geometry_df["全幅"] * geometry_df["全高"]
geometry_df["トレッド幅"] = geometry_df["トレッド幅"] * geometry_df["全幅"] * geometry_df["全高"]
geometry_df["バンパー下端幅"] = geometry_df["バンパー下端幅"] * geometry_df["全幅"] * geometry_df["全高"]
geometry_df["バンパー上端幅"] = geometry_df["バンパー上端幅"] * geometry_df["全幅"] * geometry_df["全高"]
geometry_df["ルーフ厚み"] = geometry_df["ルーフ厚み"] * geometry_df["全高"]
geometry_df["キャビン厚み"] = geometry_df["キャビン厚み"] * geometry_df["全高"]
geometry_df["ショルダー厚み"] = geometry_df["ショルダー厚み"] * geometry_df["全高"]
geometry_df["フード厚み"] = geometry_df["フード厚み"] * geometry_df["全高"]
geometry_df["バンパー厚み"] = geometry_df["バンパー厚み"] * geometry_df["全高"]

merge_df = (
    pd.merge(merge_df, geometry_df, on="file_name", how="left")
    .dropna()
    .reset_index(drop=True)
)

# 0~1にmin-max scaling
for col in geometry_columns:
    merge_df[col] = (merge_df[col] - merge_df[col].min()) / (
        merge_df[col].max() - merge_df[col].min()
    )

# loading carfolder2latent_idx
with open("../../outputs/data-split/carfolder2latent_idx.pickle", "rb") as f:
    carfolder2latent_idx = pickle.load(f)

In [3]:
# loading latent data



latent_codes = torch.load("../../outputs/latent-codes/latent_codes.pth")
latent_walker = WalkEffKANMulti(
    attribute_dim=len(geometry_columns), latent_code_dim=cfg.latent_code_dim
).to(cfg.device)
# latent_walker.load_state_dict(torch.load("../../outputs/models/ori_models/geometry_walker.pth"))
latent_walker.load_state_dict(torch.load("../../outputs_m/models/geometry_walker_256.pth"))

deepsdf = DeepSDF(
    hidden_dim=cfg.hidden_dim,
    xyz_pos_enc_dim=cfg.xyz_pos_enc_dim,
    latent_code_dim=cfg.latent_code_dim,
).to(cfg.device)
deepsdf.load_state_dict(torch.load("../../outputs_m/models/deepsdf_256.pth"))

<All keys matched successfully>

In [4]:
edit_car_info = merge_df.iloc[0]
edit_car_info

file_name       Audi_Q4_e-tron_S-line_2021
folder_name     004_AU_Q4_etron_Sline_2021
ルーフ長さ                             0.500232
キャビン長さ                            0.828888
フード長さ                              0.27887
L2                                  0.4505
ホイールベース                           0.224445
基調ライン基点長さ                         0.439544
前輪-Lの長さ                           0.246025
フロントオーバーハング                       0.503098
リアオーバーハング                         0.760155
RR-OH/FR-OH                       0.760155
H1                                0.128954
全高                                0.174541
ノーズ高さ                             0.246911
ノーズスラント量                          0.495833
基調ライン基点高さ                         0.223567
ベルトライン高さ                          0.189521
フロントバンパー下端高さ                      0.181181
リアバンパー下端高さ                        0.283851
サイドシル下端高さ                         0.186453
基調ライン角                            0.521137
FRウィンドウ傾斜角                        0.127719
RRウィンドウ傾斜角 

In [5]:
from src.utils.mesh_utils import create_mesh
from ipywidgets import interact_manual, FloatSlider


latent_code = (
    latent_codes[carfolder2latent_idx[edit_car_info["folder_name"]]]
    .reshape(1, -1)
    .to(cfg.device)
)

base_geometry = edit_car_info[geometry_columns].values


@interact_manual
def create_3d(
    lambda_=FloatSlider(value=3, min=0, max=6, step=1),
    ルーフ長さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    キャビン長さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    フード長さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    L2=FloatSlider(value=0, min=-1, max=1, step=0.1),
    ホイールベース=FloatSlider(value=0, min=-1, max=1, step=0.1),
    基調ライン基点長さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    前輪_Lの長さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    フロントオーバーハング=FloatSlider(value=0, min=-1, max=1, step=0.1),
    リアオーバーハング=FloatSlider(value=0, min=-1, max=1, step=0.1),
    RR_OH_FR_OH=FloatSlider(value=0, min=-1, max=1, step=0.1),
    H1=FloatSlider(value=0, min=-1, max=1, step=0.1),
    全高=FloatSlider(value=0, min=-1, max=1, step=0.1),
    ノーズ高さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    ノーズスラント量=FloatSlider(value=0, min=-1, max=1, step=0.1),
    基調ライン基点高さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    ベルトライン高さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    フロントバンパー下端高さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    リアバンパー下端高さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    サイドシル下端高さ=FloatSlider(value=0, min=-1, max=1, step=0.1),
    基調ライン角=FloatSlider(value=0, min=-1, max=1, step=0.1),
    FRウィンドウ傾斜角=FloatSlider(value=0, min=-1, max=1, step=0.1),
    RRウィンドウ傾斜角=FloatSlider(value=0, min=-1, max=1, step=0.1),
    全幅=FloatSlider(value=0, min=-1, max=1, step=0.1),
    キャビン幅=FloatSlider(value=0, min=-1, max=1, step=0.1),
    ルーフ幅=FloatSlider(value=0, min=-1, max=1, step=0.1),
    トレッド幅=FloatSlider(value=0, min=-1, max=1, step=0.1),
    バンパー下端幅=FloatSlider(value=0, min=-1, max=1, step=0.1),
    バンパー上端幅=FloatSlider(value=0, min=-1, max=1, step=0.1),
    ルーフ厚み=FloatSlider(value=0, min=-1, max=1, step=0.1),
    キャビン厚み=FloatSlider(value=0, min=-1, max=1, step=0.1),
    ショルダー厚み=FloatSlider(value=0, min=-1, max=1, step=0.1),
    フード厚み=FloatSlider(value=0, min=-1, max=1, step=0.1),
    バンパー厚み=FloatSlider(value=0, min=-1, max=1, step=0.1),
):
    # main-process

    delta_geometry = np.array(
        [
            ルーフ長さ,
            キャビン長さ,
            フード長さ,
            L2,
            ホイールベース,
            基調ライン基点長さ,
            前輪_Lの長さ,
            フロントオーバーハング,
            リアオーバーハング,
            RR_OH_FR_OH,
            H1,
            全高,
            ノーズ高さ,
            ノーズスラント量,
            基調ライン基点高さ,
            ベルトライン高さ,
            フロントバンパー下端高さ,
            リアバンパー下端高さ,
            サイドシル下端高さ,
            基調ライン角,
            FRウィンドウ傾斜角,
            RRウィンドウ傾斜角,
            全幅,
            キャビン幅,
            ルーフ幅,
            トレッド幅,
            バンパー下端幅,
            バンパー上端幅,
            ルーフ厚み,
            キャビン厚み,
            ショルダー厚み,
            フード厚み,
            バンパー厚み,
        ]
    )

    edit_geometry = base_geometry + delta_geometry
    edit_geometry = np.clip(edit_geometry, 0, 1)
    delta = edit_geometry - base_geometry


    changed_attributes = []
    for i, change in enumerate(delta):
        if change != 0:
            changed_attributes.append(f"{geometry_columns[i]}_{base_geometry[i]:.3f}_{edit_geometry[i]:.3f}")


    folder_name = "_".join(changed_attributes)
    os.makedirs(folder_name, exist_ok=True)  # 创建子文件夹



    print("ルーフ長さ: {:.3f} => {:.3f}".format(base_geometry[0], edit_geometry[0]))
    print("キャビン長さ: {:.3f} => {:.3f}".format(base_geometry[1], edit_geometry[1]))
    print("フード長さ: {:.3f} => {:.3f}".format(base_geometry[2], edit_geometry[2]))
    print("L2: {:.3f} => {:.3f}".format(base_geometry[3], edit_geometry[3]))
    print("ホイールベース: {:.3f} => {:.3f}".format(base_geometry[4], edit_geometry[4]))
    print("基調ライン基点長さ: {:.3f} => {:.3f}".format(base_geometry[5], edit_geometry[5]))
    print("前輪-Lの長さ: {:.3f} => {:.3f}".format(base_geometry[6], edit_geometry[6]))
    print("フロントオーバーハング: {:.3f} => {:.3f}".format(base_geometry[7], edit_geometry[7]))
    print("リアオーバーハング: {:.3f} => {:.3f}".format(base_geometry[8], edit_geometry[8]))
    print("RR-OH/FR-OH: {:.3f} => {:.3f}".format(base_geometry[9], edit_geometry[9]))
    print("H1: {:.3f} => {:.3f}".format(base_geometry[10], edit_geometry[10]))
    print("全高: {:.3f} => {:.3f}".format(base_geometry[11], edit_geometry[11]))
    print("ノーズ高さ: {:.3f} => {:.3f}".format(base_geometry[12], edit_geometry[12]))
    print("ノーズスラント量: {:.3f} => {:.3f}".format(base_geometry[13], edit_geometry[13]))
    print("基調ライン基点高さ: {:.3f} => {:.3f}".format(base_geometry[14], edit_geometry[14]))
    print("ベルトライン高さ: {:.3f} => {:.3f}".format(base_geometry[15], edit_geometry[15]))
    print("フロントバンパー下端高さ: {:.3f} => {:.3f}".format(base_geometry[16], edit_geometry[16]))
    print("リアバンパー下端高さ: {:.3f} => {:.3f}".format(base_geometry[17], edit_geometry[17]))
    print("サイドシル下端高さ: {:.3f} => {:.3f}".format(base_geometry[18], edit_geometry[18]))
    print("基調ライン角: {:.3f} => {:.3f}".format(base_geometry[19], edit_geometry[19]))
    print("FRウィンドウ傾斜角: {:.3f} => {:.3f}".format(base_geometry[20], edit_geometry[20]))
    print("RRウィンドウ傾斜角: {:.3f} => {:.3f}".format(base_geometry[21], edit_geometry[21]))
    print("全幅: {:.3f} => {:.3f}".format(base_geometry[22], edit_geometry[22]))
    print("キャビン幅: {:.3f} => {:.3f}".format(base_geometry[23], edit_geometry[23]))
    print("ルーフ幅: {:.3f} => {:.3f}".format(base_geometry[24], edit_geometry[24]))
    print("トレッド幅: {:.3f} => {:.3f}".format(base_geometry[25], edit_geometry[25]))
    print("バンパー下端幅: {:.3f} => {:.3f}".format(base_geometry[26], edit_geometry[26]))
    print("バンパー上端幅: {:.3f} => {:.3f}".format(base_geometry[27], edit_geometry[27]))
    print("ルーフ厚み: {:.3f} => {:.3f}".format(base_geometry[28], edit_geometry[28]))
    print("キャビン厚み: {:.3f} => {:.3f}".format(base_geometry[29], edit_geometry[29]))
    print("ショルダー厚み: {:.3f} => {:.3f}".format(base_geometry[30], edit_geometry[30]))
    print("フード厚み: {:.3f} => {:.3f}".format(base_geometry[31], edit_geometry[31]))
    print("バンパー厚み: {:.3f} => {:.3f}".format(base_geometry[32], edit_geometry[32]))

    n_divide = 10
    file_names = [f"{folder_name}/{i}" for i in range(n_divide)]
    divided_delta = np.linspace(0, delta, n_divide)

    for i, delta in tqdm(enumerate(divided_delta)):
        delta = (
            torch.from_numpy(delta.astype(np.float32))
            .float()
            .to(cfg.device)
            .reshape(1, -1)
        )
        edited_latent = latent_walker(
            latent_code, delta.to(cfg.device), lambda_=lambda_
        )
        edited_latent = torch.tensor(edited_latent).float().to(cfg.device)
        create_mesh(
            deepsdf,
            edited_latent,
            "{}/{}".format(folder_name, file_names[i]),  # Updated path,
            N=512,
            max_batch=80**3,
            offset=None,
            scale=None,
        )

    # return-process
    return "created 3d model!"

interactive(children=(FloatSlider(value=3.0, description='lambda_', max=6.0, step=1.0), FloatSlider(value=0.0,…